In [2]:
import pandas as pd
import sqlite3

In [11]:
con = sqlite3.connect('rusa.db')
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", con)

,name
0,riders
1,event_details
2,events
3,event_results
4,clubs


In [12]:
q = \
"""
select * 
from
    event_results er
    join event_details ed
        on er.eid = ed.eid
    join riders r
        on er.rusa = r.rusa
limit 10"""
pd.read_sql(q, con)

,cert,rusa,eid,hours,minutes,medal,finish_minutes,eid,finishers,dnf,starters,rusa,acp_code,name,first_name,last_name
0,373131,8037,5426,9,44,None,584,5426,29,0.0,29.0,8037,905014,"AGATEP, Scott",Scott,AGATEP
1,373132,6328,5426,9,20,Y,560,5426,29,0.0,29.0,6328,905014,"ALBRIGHT, Dean",Dean,ALBRIGHT
2,373133,5776,5426,11,20,Y,680,5426,29,0.0,29.0,5776,905014,"AMEEN, Sol",Sol,AMEEN
3,373134,6151,5426,8,47,None,527,5426,29,0.0,29.0,6151,905030,"ANDERSEN, Carl",Carl,ANDERSEN
4,373135,8313,5426,8,27,None,507,5426,29,0.0,29.0,8313,905048,"BACKMAN, Steve C",Steve C,BACKMAN
5,373136,8334,5426,8,0,None,480,5426,29,0.0,29.0,8334,905095,"BEEDE, Ty Thomas",Ty Thomas,BEEDE
6,373137,7372,5426,10,28,None,628,5426,29,0.0,29.0,7372,905106,"BRAHMAN, Manoucher",Manoucher,BRAHMAN
7,373138,7,5426,11,45,Y,705,5426,29,0.0,29.0,7,905106,"BRYANT, Bill",Bill,BRYANT
8,373139,5909,5426,11,20,Y,680,5426,29,0.0,29.0,5909,905014,"GALLO, Linda Dal",Linda Dal,GALLO
9,373140,6578,5426,9,35,Y,575,5426,29,0.0,29.0,6578,905014,"DICKEY, Darell D.",Darell D.,DICKEY


### Compare me with another rider

In [22]:
RUSA_A = 11548
RUSA_B = 12806

In [18]:
MEGA_TABLE_SQL = \
"""
select
    e.type,
    e.date,
    c.club_name,
    e.route,
    e.distance,
    rd.rusa,
    rd.name,
    er.cert,
    er.finish_minutes
from
    events e
    join clubs c
        on e.host_acp_code = c.acp_code
    join event_results er
        on e.eid = er.eid
    join event_details ed
        on er.eid = ed.eid
    join riders rd
        on er.rusa = rd.rusa"""

In [24]:
q = \
f"""
with 
    mt as ({MEGA_TABLE_SQL}),
    rusa_a as (
        select *
        from mt
        where mt.rusa = {RUSA_A}
    ),
    rusa_b as (
        select *
        from mt
        where mt.rusa = {RUSA_B}
    )

select
    mt.type, mt.cert, mt.date, mt.club_name, mt.distance, mt.route,
    rusa_a.rusa rusa_a, rusa_a.name name_a, rusa_a.finish_minutes as mins_a,
    rusa_b.rusa rusa_b, rusa_b.name name_b, rusa_b.finish_minutes as mins_b
from
    mt
    left join rusa_a on mt.cert = rusa_a.cert
    left join rusa_b on mt.cert = rusa_b.cert
where
    rusa_a.rusa is not null and
    rusa_b.rusa is not null
"""
pd.read_sql(q, con)

,type,cert,date,club_name,distance,route,rusa_a,name_a,mins_a,rusa_b,name_b,mins_b


In [25]:
q = \
f"""
select 
    er.rusa, r.name, floor(e.distance/100)*100 as norm_dist,
    count(*) as events,
    avg(finish_minutes)/60 as avg_hrs
    
from 
    event_results er
    join events e on er.eid = e.eid 
    join riders r on er.rusa = r.rusa 
where 
    e.type <> 'RUSAB'
    and er.rusa in ({RUSA_A}, {RUSA_B})
group by 1,2,3

"""
pd.read_sql(q, con)

,rusa,name,norm_dist,events,avg_hrs
0,11548,"SREENIVASAN, Arvi",100.0,3,5.611111
1,11548,"SREENIVASAN, Arvi",200.0,6,10.316667
2,11548,"SREENIVASAN, Arvi",300.0,3,16.366667
3,11548,"SREENIVASAN, Arvi",400.0,2,20.825000
4,11548,"SREENIVASAN, Arvi",600.0,2,37.866667
5,12806,"FRENCH, John D",100.0,7,6.361905
6,12806,"FRENCH, John D",200.0,12,11.072222
7,12806,"FRENCH, John D",300.0,4,17.179167
8,12806,"FRENCH, John D",400.0,2,22.058333
9,12806,"FRENCH, John D",600.0,2,38.225000


In [21]:
q = \
f"""
select *
from riders
where name like '%FRENCH, JOHN%'
"""
pd.read_sql(q, con)

,rusa,acp_code,name,first_name,last_name
0,12806,905030,"FRENCH, John D",John D,FRENCH
